## Imports

In [ ]:
import duckdb
import networkx as nx
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import igraph as ig
import leidenalg

from collections import defaultdict
from datetime import datetime
from geopy.distance import great_circle
from pyvis.network import Network
from shapely import wkt
import shapely.geometry

import geopandas as gpd
import matplotlib.colors as mcolors

from networkx.algorithms.community import quality
import pickle


## Constants

In [ ]:
X_SHOP_PHYSICAL_BRANDS_PCT = 'RELATED_CROSS_SHOPPING_PHYSICAL_BRANDS_PCT'
X_SHOP_DIMENSIONS = ['physical']
X_SHOP_DIMENSIONS_ALL = ['physical']
LOCATION_NAME = 'LOCATION_NAME'
TOP_CATEGORY = 'TOP_CATEGORY'
RAW_NUM_CUSTOMERS = 'RAW_NUM_CUSTOMERS'
RAW_NUM_TRANSACTIONS = 'RAW_NUM_TRANSACTIONS'
RAW_TOTAL_SPEND = 'RAW_TOTAL_SPEND'
POLYGON = 'POLYGON_WKT'

SPEND_PLACES_PATH = 'data/san-diego-county-places-spend.parquet' # added based on joined output

## Load Data

In [ ]:
r1 = duckdb.read_parquet(SPEND_PLACES_PATH)
df = r1.to_df()
df.head()

In [ ]:
df.columns

In [ ]:
def parse_json_to_tuple(json_str: str) -> list[tuple[str, int]]:
    if not json_str:
        return None
    try:
        data_dict = json.loads(json_str)

        # Handle the nested structure
        if isinstance(data_dict, dict) and "key_value" in data_dict:
            return [(item["key"], item["value"]) for item in data_dict["key_value"]]

        # If it's already a flat dict
        elif isinstance(data_dict, dict):
            return [(k, v) for k, v in data_dict.items()]

        # If it's a list of dicts already
        elif isinstance(data_dict, list):
            return [(item["key"], item["value"]) for item in data_dict]

        return None
    except (ValueError, SyntaxError, KeyError, TypeError):
        return None

## Parse Related Cross Shopping Column

In [ ]:
df['parsed_physical_brands'] = df[X_SHOP_PHYSICAL_BRANDS_PCT].apply(parse_json_to_tuple)

## Set Polygon Column for Maps

In [ ]:
df["GEOMETRY"] = df['POLYGON_WKT'].apply(lambda x: wkt.loads(x) if pd.notna(x) else None)

## Construct the graph

### Cross shopping implementation

This implementation accounts for location as nodes and uses cross shopping brands as edges.

Notes:
-  First builds brand index for effeciency, such that every location_node has an associated brand. We use location name as the brand, as not all locations have associated brands from SafeGraph but attributed brand names equal to brand_name. Structure in comments. 
- 

In [ ]:
# intialize graphs
graphs = {
    "physical": nx.DiGraph()
}

df["brand_sets"] = df.apply(lambda row: {
    "physical":     row["parsed_physical_brands"]
}, axis=1)

In [ ]:
# build brand-to-locations index for faster lookup
# builds an index like:
# {
#     "physical": {
#         "Target": [
#             "Target (33.123, -117.456)",
#             "Target (34.789, -118.123)"
#         ],
#         "Walmart": [
#             "Walmart (33.456, -117.789)"...


def build_brand_index(graphs_dict):
    """Build index: brand -> list of node_ids with that brand"""
    brand_index = {}
    for graph_name, G in graphs_dict.items():
        brand_index[graph_name] = defaultdict(list)
        for node_id in G.nodes():
            brand = G.nodes[node_id].get('brand')
            if brand:
                brand_index[graph_name][brand].append(node_id)
    return brand_index

# add all nodes with location name as brand
def add_nodes(row):
    location = row[LOCATION_NAME]
    latitude = row["LATITUDE"]
    longitude = row["LONGITUDE"]
    node_id = f"{location} ({latitude:.6f}, {longitude:.6f})"
    
    for g in graphs.values():
        g.add_node(
            node_id,
            name=location,
            brand=location,
            label=location,
            category=row[TOP_CATEGORY],
            num_customers=row[RAW_NUM_CUSTOMERS],
            num_transactions=row[RAW_NUM_TRANSACTIONS],
            total_spend=row[RAW_TOTAL_SPEND],
            latitude=latitude,
            longitude=longitude,
            geometry_wkt=row["POLYGON_WKT"],
        )

df.apply(add_nodes, axis=1)

# with our graphs and nodes built, we can use the function above to build the brand index
brand_index = build_brand_index(graphs)

# now we can add edges using the brand index
def add_cross_shopping_edges(row):
    location_or_brand = row[LOCATION_NAME]
    latitude = row["LATITUDE"]
    longitude = row["LONGITUDE"]
    source_node_id = f"{location_or_brand} ({latitude:.6f}, {longitude:.6f})"
    
    for graph_name, brand_list in row["brand_sets"].items(): # this gets our cross shopping dimension brands
        if not brand_list:
            continue
        
        G = graphs[graph_name]
        
        for brand, pct in brand_list: # brand list is a list of tuples with brand name and percentage
            weight = pct / 100
            if weight > 0.25: # bumping this up to 25% to reduce noise
                # we use index to find target nodes quickly, reference structure above for clarification
                # this is a similar implementation to the original implementation we are just now 
                # mapping brand to location nodes

                # this retrieves the list of location nodes that have the brand (location name eg Walmart, Starbucks)
                # and returns a list of node ids which includes the location name and coordinates
                target_nodes = brand_index[graph_name].get(brand, [])
                
                # now we iterate through the target nodes and add edges
                for target_node_id in target_nodes:
                    # we don't want to add an edge to itself
                    if target_node_id != source_node_id:
                        # if the brand already exists, we aggregate the weight
                        # this occurs because we may have multiple location nodes for the same brand
                        # so we take the max of the weights
                        # conversly, we don't know the exact location of the brand, so we use the max

                        # practically - this checks if "Walmart" is already in the graph (remember this is now our source node)
                        # and if so, it aggregates the weight of the edge
                        if G.has_edge(source_node_id, target_node_id):
                            G[source_node_id][target_node_id]['weight'] = max(
                                G[source_node_id][target_node_id].get('weight', 0), 
                                weight
                            )
                        else:
                            G.add_edge(source_node_id, target_node_id, weight=weight)

df.apply(add_cross_shopping_edges, axis=1)

### Continue brand implementation

In [ ]:
# check graph nodes and edge counts
print('Physical: ', graphs['physical'].number_of_nodes(), graphs['physical'].number_of_edges())

In [ ]:
# filter out isolated nodes
for name, G in graphs.items():
    isolated_nodes = list(nx.isolates(G))
    G.remove_nodes_from(isolated_nodes)
    print(f"{name} graph after removing isolates: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

## Export Graph to GEXF

In [ ]:
# export the graph to a GEXF file for visualization in Gephi
def export_graph(G: nx.DiGraph, name: str):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f"graphs/{name}_graph_{timestamp}.gexf"
    nx.write_gexf(G, output_file)

for name, G in graphs.items():
    export_graph(G, name)

## Notes
Overview
- physical graph, will use this for all downstream (provide full end point to businesses)
- louvain is baseline, laiden is primary model
- Goal/Eval: Top N locations to provide to business / community detection

## Clustering

### Global Clustering Coefficient

 >The global clustering coefficient (aka "transitivity") is based on triplets of nodes. A triplet consists of three connected nodes. A triangle therefore includes three closed triplets, one centered on each of the nodes (n.b. this means the three triplets in a triangle come from overlapping selections of nodes). The global clustering coefficient is the number of closed triplets (or 3 x triangles) over the total number of triplets (both open and closed). The first attempt to measure it was made by Luce and Perry (1949). This measure gives an indication of the clustering in the whole network (global), and can be applied to both undirected and directed networks.

tldr: transitivity ranges from 0 to 1 and can be interpreted as: the probability that two nodes that share a neighbor are themselves connected.

 GeeksforGeeks. (2022, October 31). Clustering coefficient in graph Theory. GeeksforGeeks. https://www.geeksforgeeks.org/dsa/clustering-coefficient-graph-theory/

#### NOTE:
Adjusted this slightly. Now utilizes an appoximate method (often yields similar results) using with node and edge increase with location based method. We will also save output to pickle/dataframe

In [ ]:
def compute_global_clustering_coeff(G: nx.DiGraph):
    """Compute global clustering with optimization for large graphs."""
    UG = G.to_undirected()
    
    # for the physical graph, we use the approximation method, too large to compute exactly
    if G.number_of_edges() > 500000:
        print(f"  Graph has {G.number_of_edges()} edges - using approximation...")
        # sample nodes for faster computation
        nodes = list(UG.nodes())
        if len(nodes) > 5000:
            import random
            sample_nodes = random.sample(nodes, 5000)
            local_clust = nx.clustering(UG, nodes=sample_nodes)
        else:
            local_clust = nx.clustering(UG)
        return np.mean(list(local_clust.values()))
    else:
        # we will exact computation for smaller graphs if relevant later
        return nx.transitivity(UG)

for name, G in graphs.items():
    print(f"Computing clustering for {name} graph ({G.number_of_edges()} edges)...")
    gcc = compute_global_clustering_coeff(G)
    print(f"Global Clustering Coefficient for {name} graph: {gcc}")

In [ ]:
# Collect results in a list
results = []

for name, G in graphs.items():
    print(f"Computing clustering for {name} graph ({G.number_of_edges()} edges)...")
    gcc = compute_global_clustering_coeff(G)
    print(f"Global Clustering Coefficient for {name} graph: {gcc}")
    
    results.append({
        'graph_name': name,
        'clustering_coefficient': gcc,
        'num_nodes': G.number_of_nodes(),
        'num_edges': G.number_of_edges(),
        'method': 'approximation' if G.number_of_edges() > 500000 else 'exact'
    })

# Create DataFrame and save
clustering_df = pd.DataFrame(results)
clustering_df.to_csv('clustering_results.csv', index=False)
clustering_df.to_parquet('clustering_results.parquet', index=False)  # More efficient

print(f"\nResults saved:")
print(clustering_df)

In [ ]:
# def compute_global_clustering_coeff(G: nx.DiGraph):
#     # convert digraph to undirected graph
#     UG = G.to_undirected()
#     return nx.transitivity(UG)

# for name, G in graphs.items():
#     gcc = compute_global_clustering_coeff(G)
#     print(f"Global Clustering Coefficient for {name} graph: {gcc}")

In [ ]:
# def compute_pagerank(G: nx.DiGraph):
#     pagerank = nx.pagerank(G, alpha=0.85, weight='weight')
#     return sorted(pagerank.items(), key=lambda item: item[1], reverse=True)

# def compute_clustering(G: nx.DiGraph):
#     clusters = nx.clustering(G, weight='weight')
#     return sorted(clusters.items(), key=lambda item: item[1], reverse=True)

# pagerank = compute_pagerank(graphs['physical'])
# clusters = compute_clustering(graphs['physical'])

Optimized for large graphs

In [ ]:
def compute_pagerank(G: nx.DiGraph):
    """Compute PageRank with optimization for large graphs."""
    if G.number_of_edges() > 500000:
        print(f"  Large graph: {G.number_of_edges()} edges - using optimized settings...")
        pagerank = nx.pagerank(G, alpha=0.85, weight='weight', 
                              max_iter=50, tol=1e-5)
    else:
        pagerank = nx.pagerank(G, alpha=0.85, weight='weight')
    return sorted(pagerank.items(), key=lambda item: item[1], reverse=True)

# Use optimized versions
print("Computing PageRank for physical graph...")
pagerank = compute_pagerank(graphs['physical'])
print(f"✓ PageRank complete: {len(pagerank)} nodes")

Skip additional clustering for now

In [ ]:
# def compute_clustering(G: nx.DiGraph):
#     """Compute clustering with sampling for large graphs."""
#     if G.number_of_edges() > 500000:
#         print(f"  Large graph: {G.number_of_edges()} edges - sampling nodes...")
#         nodes = list(G.nodes())
#         if len(nodes) > 2000:
#             import random
#             sample_nodes = random.sample(nodes, 2000)
#             clusters = nx.clustering(G, nodes=sample_nodes, weight='weight')
#         else:
#             clusters = nx.clustering(G, weight='weight')
#     else:
#         clusters = nx.clustering(G, weight='weight')
#     return sorted(clusters.items(), key=lambda item: item[1], reverse=True)


# print("\nComputing clustering for physical graph...")
# clusters = compute_clustering(graphs['physical'])
# print(f"✓ Clustering complete: {len(clusters)} nodes")

## Degree Analysis

In [ ]:
# find most influential nodes using out_degree 
def plot_in_out_degree(G: nx.DiGraph, related_x_shop_type: str):
    in_degrees = dict(G.in_degree())
    out_degrees = dict(G.out_degree())

    plt.scatter(in_degrees.values(), out_degrees.values())
    plt.xlabel("In-degree")
    plt.ylabel("Out-degree")
    plt.title("In vs Out Degree Distribution ({})".format(related_x_shop_type))
    plt.show()

plot_in_out_degree(graphs['physical'], 'physical')

In [ ]:
# find the top n nodes with the highest out_degree
def top_out_degree_nodes(num_nodes: int, G: nx.DiGraph) -> list[tuple[str, int]]:
    out_degrees = dict(G.out_degree())
    return sorted(out_degrees.items(), key=lambda item: item[1], reverse=True)[:num_nodes]

top10_out_physical = top_out_degree_nodes(10, graphs['physical'])

for dimension in X_SHOP_DIMENSIONS:
    print(f"Top 10 out-degree nodes for {dimension} graph:")
    top10_out = top_out_degree_nodes(10, graphs[dimension])
    for node, degree in top10_out:
        print(f"{node}: {degree}")

In [ ]:
# find the top n nodes with the highest in_degree
def top_in_degree_nodes(num_nodes: int, G: nx.DiGraph) -> list[tuple[str, int]]:
    in_degrees = dict(G.in_degree())
    return sorted(in_degrees.items(), key=lambda item: item[1], reverse=True)[:num_nodes]

for dimension in X_SHOP_DIMENSIONS:
    print(f"Top 10 in-degree nodes for {dimension} graph:")
    top10_in = top_in_degree_nodes(10, graphs[dimension])
    for node, degree in top10_in:
        print(f"{node}: {degree}")

In [ ]:
# plot distribution of in-degrees and out-degrees
def plot_degree_distribution(G: nx.DiGraph, related_x_shop_type: str):
    in_degrees = [d for n, d in G.in_degree()]
    out_degrees = [d for n, d in G.out_degree()]

    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.hist(in_degrees, bins=30, color='blue', alpha=0.7, log=True)
    plt.xlabel("In-degree")
    plt.ylabel("Frequency")
    plt.title("In-degree Distribution ({})".format(related_x_shop_type))

    plt.subplot(1, 2, 2)
    plt.hist(out_degrees, bins=30, color='green', alpha=0.7, log=True)
    plt.xlabel("Out-degree")
    plt.ylabel("Frequency")
    plt.title("Out-degree Distribution ({})".format(related_x_shop_type))

    plt.tight_layout()
    plt.show()

plot_degree_distribution(graphs['physical'], 'physical')

### Degree Neighbors

In [ ]:
# find all nodes that have an incoming edge into a specific node
def first_degree_in_neighbors(node: str, G: nx.DiGraph) -> set[str]:
    """
    Return all 1st-degree neighbors that have an incoming edge into `node`.
    """
    if node not in G:
        return set()
    return set(G.predecessors(node))

def first_degree_out_neighbors(node: str, G: nx.DiGraph) -> set[str]:
    """
    Return the 1st-degree neighbors that have an outgoing edge from `node`.
    """
    if node not in G:
        return set()
    return set(G.successors(node))

def second_degree_in_neighbors(node: str, G: nx.DiGraph) -> set[str]:
    """
    Return the 2nd-degree neighbors that have an incoming edge into `node`.
    Excludes the node itself and direct 1st-degree neighbors.
    """
    if node not in G:
        return set()
    first = set(G.successors(node))
    second = set()
    for n in first:
        second.update(G.successors(n))
    second -= first
    second.discard(node)
    return second

In [ ]:
first_degree_in_neighbors("Tilly's (32.768785, -117.147968)", graphs['physical'])

## Geo-Filters 

We define a function that allows us to specify a radius prior to community detection. We aim to evaluate various cross-shopping communities with a geographic filter (radius in miles). This uses the intial graph created and provides a subgraph based on this determination. We can use either a center location for broader analyis, or a specific location in our dataset.

First, we define a helper function to filer the nodes based on the set radius in miles. Then, we define a function that utlizes the subgraph method from NetworkX to filter based on these filtered nodes. We also use this opportunity to return only connected brands as a parameter to further filter the graph.

In [ ]:
# helper function to get nodes within a radius
def get_nodes_within_radius(G, center_node, radius_mi):
    """
    Get all nodes within a given radius (in km) of a center node.
    Uses geodesic distance for accuracy.
    """
    if center_node not in G:
        return []
    
    center_lat = G.nodes[center_node].get('latitude')
    center_lon = G.nodes[center_node].get('longitude')
    
    if center_lat is None or center_lon is None:
        print(f"Warning: {center_node} does not have latitude/longitude data")
        return []
    
    center_point = (center_lat, center_lon)
    nodes_within_radius = []
    
    for node in G.nodes():
        node_lat = G.nodes[node].get('latitude')
        node_lon = G.nodes[node].get('longitude')
        
        if node_lat is None or node_lon is None:
            continue
        
        node_point = (node_lat, node_lon)
        distance = great_circle(center_point, node_point).miles
        
        if distance <= radius_mi:
            nodes_within_radius.append(node)
    
    nodes_within_radius.sort(key=lambda x: x[1])
    return nodes_within_radius

In [ ]:
def filter_graph_by_radius(G: nx.DiGraph, center_node: str, radius_mi: float, 
                           include_connected_brands: bool = True) -> nx.DiGraph:
    """
    Filter a graph to include only nodes within a specified radius of a center node.
    Optionally includes brand nodes connected to location nodes within the radius.
    
    Parameters:
    -----------
    G : NetworkX DiGraph
        The original graph
    center_node : str
        The center node (location) to filter around
    radius_mi : float
        Radius in miles
    include_connected_brands : bool, default=True
        If True, includes brand nodes connected to location nodes within radius.
        If False, only includes location nodes within radius.
    
    Returns:
    --------
    NetworkX DiGraph : Filtered subgraph containing only nodes within radius
    """
    if center_node not in G:
        print(f"Warning: {center_node} not found in graph")
        return nx.DiGraph()
    
    # Get location nodes within radius (nodes with lat/lon)
    location_nodes_within_radius = get_nodes_within_radius(G, center_node, radius_mi)
    
    if not location_nodes_within_radius:
        print(f"No nodes found within {radius_mi} miles of {center_node}")
        return nx.DiGraph()
    
    # Set of nodes to include in filtered graph
    nodes_to_include = set(location_nodes_within_radius)
    
    # Optionally include brand nodes connected to location nodes within radius
    if include_connected_brands:
        for location in location_nodes_within_radius:
            # Get all neighbors (brands) connected to this location
            neighbors = list(G.successors(location)) + list(G.predecessors(location))
            nodes_to_include.update(neighbors)
    
    # Create subgraph with only the selected nodes
    filtered_graph = G.subgraph(nodes_to_include).copy()
    
    print(f"Filtered graph: {filtered_graph.number_of_nodes()} nodes, "
          f"{filtered_graph.number_of_edges()} edges "
          f"(from {G.number_of_nodes()} nodes, {G.number_of_edges()} edges)")
    
    return filtered_graph

## Community Detection

### Louvain Communities

A Louvain community is a group of nodes in a network that are more densely connected to each other than to the rest of the network, identified by the Louvain algorithm through greedy modularity maximization. The algorithm iteratively moves nodes between communities to increase modularity and aggregates communities hierarchically until no further improvement is possible, producing a partition that reveals the network’s latent structure. In the context of a business cross-shopping network, Louvain communities correspond to groups of stores that share many customers relative to the broader network, highlighting patterns of customer behavior that may not align with obvious categories (Blondel et al., 2008).

Citation:
Blondel, V. D., Guillaume, J.-L., Lambiotte, R., & Lefebvre, E. (2008). Fast unfolding of communities in large networks. Journal of Statistical Mechanics: Theory and Experiment, 2008(10), P10008. https://doi.org/10.1088/1742-5468/2008/10/P10008

In [ ]:
def get_louvain_communities(G: nx.DiGraph, weight='weight', seed=47):
    """Detect communities in the graph using the Louvain method."""
    communities = nx.community.louvain_communities(G, weight=weight, seed=seed)

    return communities

In [ ]:
# compute Louvain communities for each graph
louvain_communities_physical = get_louvain_communities(graphs['physical'])

# size of communities in each graph
print(len(louvain_communities_physical))

### Leiden Communities
Leiden communities are groups of nodes that are densely connected internally and sparsely connected externally, identified by the Leiden algorithm. The algorithm iteratively moves nodes between communities to maximize a quality function (e.g., modularity), refines communities to ensure internal connectivity, and aggregates communities hierarchically until no further improvement is possible. Compared to Louvain, Leiden produces more stable, internally connected, and interpretable communities, making it particularly useful for real-world networks such as cross-shopping networks where loosely connected nodes and disconnected subgroups are common (Traag et al., 2019).

Traag, V. A., Waltman, L., & van Eck, N. J. (2019). From Louvain to Leiden: guaranteeing well-connected communities. Scientific Reports, 9, 5233. https://arxiv.org/abs/1810.08473

In [ ]:
def convert_to_igraph(G: nx.DiGraph) -> tuple[ig.Graph, list]:
    """
    Convert a NetworkX DiGraph to an iGraph Graph.
    Returns the graph and a list of node IDs in the same order as iGraph vertices.
    """
    ig_graph = ig.Graph.from_networkx(G)
    
    # Get the node ids in the same order as iGraph vertices preserves the node order
    nx_node_ids = list(G.nodes())
    
    return ig_graph, nx_node_ids

# Convert membership to list of sets
def leiden_to_sets(graph, partition, nx_node_ids):
    """
    Convert Leiden partition to list of sets of node IDs.
    
    Parameters:
    -----------
    graph : ig.Graph
        The iGraph graph
    partition : leidenalg.RBConfigurationVertexPartition
        The partition object
    nx_node_ids : list
        List of NetworkX node IDs in the same order as iGraph vertices
    """
    communities = defaultdict(set)
    
    # Map partition membership to NetworkX node IDs
    for i, cid in enumerate(partition.membership):
        node_id = nx_node_ids[i]  # Get the actual node_id from NetworkX
        communities[cid].add(node_id)
    
    return list(communities.values())


In [ ]:
ig_physical, nx_nodes_physical = convert_to_igraph(graphs['physical'])

In [ ]:
leiden_communities_physical = leidenalg.find_partition(graph=ig_physical,
                                                  partition_type=leidenalg.ModularityVertexPartition, 
                                                  weights='weight')


In [ ]:
leiden_set_physical = leiden_to_sets(ig_physical, leiden_communities_physical, nx_nodes_physical)

### Evaluate Community Quality

In [ ]:
def evaluate_community_quality(G: nx.DiGraph, communities: list[set[str]]):
    """Evaluate the quality of the detected communities using modularity."""
    Q = quality.modularity(G, communities)
    return Q

In [ ]:
mod = evaluate_community_quality(graphs['physical'], louvain_communities_physical)

print("Evaluating Louvain Communities")
print(f"Modularity Scores (physical): {mod}")
print()
print("Evaluating Leiden Communities")
print(f"Modularity Scores (physical): {leiden_communities_physical.modularity}")

## Mapping

We define a mapping function to display nodes in the graph, and optionally their associated community. This function returns a folium map object including:
- Lines drawn between graph edges 
- Community color coding, if community assignments are provided
- Pop ups to provide additional detail (e.g. category, total spend, customers)

We can save the map file if we specify a name

In [ ]:
def visualize_filtered_graph(G: nx.DiGraph, 
                            center_node: str = None,
                            center_lat: float = None,
                            center_lon: float = None,
                            communities: list[set[str]] = None,
                            show_edges: bool = True,
                            show_polygons: bool = False,
                            zoom_start: int = 12,
                            output_file: str = None) -> folium.Map:
    """
    Visualize filtered graph nodes on a folium map.
    
    Parameters:
    -----------
    G : NetworkX DiGraph
        The filtered graph to visualize
    center_node : str, optional
        Center node name (used to center map if center_lat/lon not provided)
    center_lat : float, optional
        Center latitude for map
    center_lon : float, optional
        Center longitude for map
    communities : list of sets, optional
        Community assignments (for color-coding nodes)
    df : pd.DataFrame, optional
        Deprecated - kept for backward compatibility. Geometry now comes from node attributes.
    show_edges : bool, default=True
        Whether to show edges between nodes as lines
    show_polygons : bool, default=False
        Whether to show polygon geometries from node attributes
    zoom_start : int, default=12
        Initial zoom level
    output_file : str, optional
        Filename to save map (e.g., 'filtered_graph_map.html')
    
    Returns:
    --------
    folium.Map : The folium map object
    """
    # Determine map center
    if center_lat is not None and center_lon is not None:
        map_center = [center_lat, center_lon]
    elif center_node and center_node in G:
        center_lat = G.nodes[center_node].get('latitude')
        center_lon = G.nodes[center_node].get('longitude')
        if center_lat and center_lon:
            map_center = [center_lat, center_lon]
        else:
            # Calculate center from all nodes
            lats = [G.nodes[n].get('latitude') for n in G.nodes() 
                   if G.nodes[n].get('latitude') is not None]
            lons = [G.nodes[n].get('longitude') for n in G.nodes() 
                   if G.nodes[n].get('longitude') is not None]
            map_center = [np.mean(lats), np.mean(lons)] if lats else [33.036986, -117.292447]
    else:
        # Calculate center from all nodes
        lats = [G.nodes[n].get('latitude') for n in G.nodes() 
               if G.nodes[n].get('latitude') is not None]
        lons = [G.nodes[n].get('longitude') for n in G.nodes() 
               if G.nodes[n].get('longitude') is not None]
        map_center = [np.mean(lats), np.mean(lons)] if lats else [33.036986, -117.292447]
    
    # Create map
    m = folium.Map(location=map_center, zoom_start=zoom_start)
    
    # Create community color mapping if communities provided
    node_to_community = {}
    if communities:
        colors = plt.cm.Set3(np.linspace(0, 1, len(communities)))
        for i, comm in enumerate(communities):
            for node in comm:
                node_to_community[node] = i
    else:
        # Default color if no communities
        colors = None
    
    # Add polygons if requested (using geometry from node attributes)
    if show_polygons:
        # Collect nodes with geometry
        features = []
        for node in G.nodes():
            geometry_wkt = G.nodes[node].get('geometry_wkt')  # Get WKT string
            if geometry_wkt is not None:
                try:
                    # Convert WKT string to Shapely geometry
                    geometry = wkt.loads(geometry_wkt)
                    
                    # Get node attributes for popup
                    category = G.nodes[node].get('category', 'N/A')
                    
                    # Convert Shapely geometry to GeoJSON format
                    geojson_geom = shapely.geometry.mapping(geometry)
                    
                    # Create GeoJSON feature
                    feature = {
                        "type": "Feature",
                        "geometry": geojson_geom,
                        "properties": {
                            "LOCATION_NAME": node,
                            "TOP_CATEGORY": category
                        }
                    }
                    features.append(feature)
                except Exception as e:
                    # Skip invalid geometries
                    continue
        
        if features:
            geojson_data = {
                "type": "FeatureCollection",
                "features": features
            }
            
            folium.GeoJson(
                geojson_data,
                style_function=lambda feature: {
                    'fillColor': 'lightblue',
                    'color': 'blue',
                    'weight': 1,
                    'fillOpacity': 0.3,
                },
                tooltip=folium.GeoJsonTooltip(fields=['LOCATION_NAME'], aliases=['Location:']),
                popup=folium.GeoJsonPopup(fields=['LOCATION_NAME', 'TOP_CATEGORY'])
            ).add_to(m)
    
    # Add edges as lines
    if show_edges:
        for u, v, data in G.edges(data=True):
            u_lat = G.nodes[u].get('latitude')
            u_lon = G.nodes[u].get('longitude')
            v_lat = G.nodes[v].get('latitude')
            v_lon = G.nodes[v].get('longitude')
            
            # Only draw edge if both nodes have coordinates (location nodes)
            if (u_lat is not None and u_lon is not None and 
                v_lat is not None and v_lon is not None):
                weight = data.get('weight', 1.0)
                # Scale line width based on weight
                line_width = max(1, min(5, weight * 10))
                
                folium.PolyLine(
                    locations=[[u_lat, u_lon], [v_lat, v_lon]],
                    weight=line_width,
                    color='gray',
                    opacity=0.05
                ).add_to(m)
    
    # Add markers for location nodes (nodes with lat/lon)
    for node in G.nodes():
        node_lat = G.nodes[node].get('latitude')
        node_lon = G.nodes[node].get('longitude')
        
        # Skip nodes without coordinates (likely brand nodes)
        if node_lat is None or node_lon is None:
            continue
        
        # Determine color based on community
        if communities and node in node_to_community:
            comm_id = node_to_community[node]
            color = mcolors.rgb2hex(colors[comm_id][:3])
            icon_color = 'white'
        else:
            color = 'blue'
            icon_color = 'white'
        
        # Get node attributes for popup
        category = G.nodes[node].get('category', 'N/A')
        num_customers = G.nodes[node].get('num_customers', 'N/A')
        total_spend = G.nodes[node].get('total_spend', 'N/A')
        
        # Create popup HTML
        popup_html = f"""
        <div style="font-family: Arial; width: 200px;">
            <h4 style="margin: 5px 0;">{node}</h4>
            <p style="margin: 2px 0;"><b>Category:</b> {category}</p>
            <p style="margin: 2px 0;"><b>Customers:</b> {num_customers}</p>
            <p style="margin: 2px 0;"><b>Total Spend:</b> ${total_spend:,.2f}</p>
        </div>
        """ if isinstance(total_spend, (int, float)) else f"""
        <div style="font-family: Arial; width: 200px;">
            <h4 style="margin: 5px 0;">{node}</h4>
            <p style="margin: 2px 0;"><b>Category:</b> {category}</p>
            <p style="margin: 2px 0;"><b>Customers:</b> {num_customers}</p>
            <p style="margin: 2px 0;"><b>Total Spend:</b> {total_spend}</p>
        </div>
        """
        
        folium.CircleMarker(
            location=[node_lat, node_lon],
            radius=6,
            popup=folium.Popup(popup_html, max_width=300),
            tooltip=node,
            color='black',
            fillColor=color,
            fillOpacity=0.7,
            weight=2
        ).add_to(m)
    
    # Add center marker if center_node specified
    if center_node and center_node in G:
        center_lat = G.nodes[center_node].get('latitude')
        center_lon = G.nodes[center_node].get('longitude')
        if center_lat and center_lon:
            folium.Marker(
                location=[center_lat, center_lon],
                popup=f"Center: {center_node}",
                tooltip=f"Center: {center_node}",
                icon=folium.Icon(color='red', icon='star', prefix='fa')
            ).add_to(m)
    
    # Save map if filename provided
    if output_file:
        m.save(output_file)
        print(f"Map saved to {output_file}")
    
    return m


# Convenience function for filtered graph visualization
def visualize_filtered_communities(filtered_G: nx.DiGraph, 
                                   communities: list[set[str]],
                                   center_node: str = None,
                                   output_file: str = None) -> folium.Map:
    """
    Visualize filtered graph with community assignments.
    
    Parameters:
    -----------
    filtered_G : NetworkX DiGraph
        The filtered graph
    communities : list of sets
        Community assignments
    center_node : str, optional
        Center node name
    df : pd.DataFrame, optional
        Deprecated - geometry now comes from node attributes
    output_file : str, optional
        Output filename
    
    Returns:
    --------
    folium.Map : The folium map
    """
    return visualize_filtered_graph(
        G=filtered_G,
        center_node=center_node,
        communities=communities,
        show_edges=True,
        show_polygons=False,
        output_file=output_file
    )

### Show example map

Below shows a complete example of capt

In [ ]:
center_node = "McDonald's (32.981937, -117.076287)"
radius_miles = 2.0
graph_type = 'physical' 

filtered_graph = filter_graph_by_radius(
    graphs[graph_type], 
    center_node=center_node, 
    radius_mi=radius_miles,
    include_connected_brands=False # can be set to false to exclude brand nodes
)

print(f"Filtered graph: {filtered_graph.number_of_nodes()} nodes, {filtered_graph.number_of_edges()} edges")

# run community detection on the filtered graph
# we first re-remove isolated nodes first in subgraph
isolated = list(nx.isolates(filtered_graph))
filtered_graph.remove_nodes_from(isolated)
print(f"After removing isolates: {filtered_graph.number_of_nodes()} nodes, {filtered_graph.number_of_edges()} edges")

# run louvain community detection for now. We can use leiden later.
# we can also use global community detection from before, but this is faster- more stable for api calls 
communities = get_louvain_communities(filtered_graph)

print(f"\nFound {len(communities)} communities")
for i, comm in enumerate(communities):
    print(f"  Community {i+1}: {len(comm)} nodes")

# visualize the filtered graph with communities
map_viz = visualize_filtered_graph(
    G=filtered_graph,
    center_node=center_node,
    communities=communities,
    show_edges=True,
    show_polygons=True, 
    zoom_start=14, 
    output_file='mcdonalds_2mile_communities.html' # change as needed
)

# Display the map
map_viz